In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import sys
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression

In [2]:
path = Path(os.getcwd())
base_dir = path.parent.parent

In [3]:
#Windows format
data_in = os.path.join(str(base_dir) , "da_data_repo\\bisnode-firms\\clean\\")
data_out = os.path.join(str(base_dir) , "da_data_repo\\bisnode-firms\\")
output = os.path.join(data_out,'output')
func = os.path.join(str(base_dir) ,   "da_case_studies\\ch00-tech-prep\\")

In [4]:
current_csv_path = os.path.join(data_out,"bisnode_firms_clean.csv")
data = pd.read_csv(current_csv_path)

In [5]:
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19036 entries, 0 to 19035
Data columns (total 118 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   year                            int64  
 1   comp_id                         float64
 2   begin                           object 
 3   end                             object 
 4   amort                           float64
 5   curr_assets                     float64
 6   curr_liab                       float64
 7   extra_exp                       float64
 8   extra_inc                       float64
 9   extra_profit_loss               float64
 10  fixed_assets                    float64
 11  inc_bef_tax                     float64
 12  intang_assets                   float64
 13  inventories                     float64
 14  liq_assets                      float64
 15  material_exp                    float64
 16  personnel_exp                   float64
 17  profit_loss_year              

In [6]:
# Define variable sets ----------------------------------------------
# (making sure we use categories)
data['ind2_cat'] = data['ind2_cat'].astype("category")
data['urban_m'] = data['urban_m'].astype("category")

rawvars = ["curr_assets", "curr_liab", "extra_exp", "extra_inc", "extra_profit_loss", "fixed_assets",
              "inc_bef_tax", "intang_assets", "inventories", "liq_assets", "material_exp", "personnel_exp",
              "profit_loss_year", "sales", "share_eq", "subscribed_cap"]
qualityvars = ["balsheet_flag", "balsheet_length", "balsheet_notfullyear"]
engvar = ["total_assets_bs", "fixed_assets_bs", "liq_assets_bs", "curr_assets_bs",
            "share_eq_bs", "subscribed_cap_bs", "intang_assets_bs", "extra_exp_pl",
            "extra_inc_pl", "extra_profit_loss_pl", "inc_bef_tax_pl", "inventories_pl",
            "material_exp_pl", "profit_loss_year_pl", "personnel_exp_pl"]
engvar2 = ["extra_profit_loss_pl_quad", "inc_bef_tax_pl_quad",
             "profit_loss_year_pl_quad", "share_eq_bs_quad"]
engvar3=[]
for col in data.columns:
    if col.endswith('flag_low') or col.endswith('flag_high') or col.endswith('flag_error') or col.endswith('flag_zero'):
        engvar3.append(col)


d1 =  ["d1_sales_mil_log_mod", "d1_sales_mil_log_mod_sq",
         "flag_low_d1_sales_mil_log", "flag_high_d1_sales_mil_log"]
hr = ["female", "ceo_age", "flag_high_ceo_age", "flag_low_ceo_age",
        "flag_miss_ceo_age", "ceo_count", "labor_avg_mod",
        "flag_miss_labor_avg", "foreign_management"]
firm = ["age", "age2", "new", "ind2_cat", "m_region_loc", "urban_m"]

# interactions for logit, LASSO
interactions1 = ["ind2_cat*age", "ind2_cat*age2",
                   "ind2_cat*d1_sales_mil_log_mod", "ind2_cat*sales_mil_log",
                   "ind2_cat*ceo_age", "ind2_cat*foreign_management",
                   "ind2_cat*female",   "ind2_cat*urban_m", "ind2_cat*labor_avg_mod"]
interactions2 = ["sales_mil_log*age", "sales_mil_log*female",
                   "sales_mil_log*profit_loss_year_pl", "sales_mil_log*foreign_management"]

X1 = ["sales_mil_log", "sales_mil_log_sq", "d1_sales_mil_log_mod", "profit_loss_year_pl", "ind2_cat"]
X2 = ["sales_mil_log", "sales_mil_log_sq", "d1_sales_mil_log_mod", "profit_loss_year_pl", "fixed_assets_bs",
      "share_eq_bs","curr_liab_bs",   "curr_liab_bs_flag_high", "curr_liab_bs_flag_error",  "age",
      "foreign_management" , "ind2_cat"]
X3 = ["sales_mil_log", "sales_mil_log_sq"] + firm + engvar + d1
X4 = ["sales_mil_log", "sales_mil_log_sq"] + firm + engvar + engvar2 + engvar3 + d1 + hr + qualityvars
X5 = ["sales_mil_log", "sales_mil_log_sq"] + firm + engvar + engvar2 + engvar3 + d1 + hr + qualityvars + interactions1 + interactions2


In [7]:
# for LASSO
logitvars = ["sales_mil_log", "sales_mil_log_sq", engvar, engvar2, engvar3, d1, hr, firm, qualityvars, interactions1, interactions2]

In [8]:
# for RF (no interactions, no modified features)
rfvars  = ["sales_mil", "d1_sales_mil_log", rawvars, hr, firm, qualityvars]

In [9]:
# Check simplest model X1
ols_modelx1 = smf.ols("default ~"+ '+'.join(X1), data = data).fit()
print(ols_modelx1.summary())

                            OLS Regression Results                            
Dep. Variable:                default   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.120
Method:                 Least Squares   F-statistic:                     236.7
Date:                Wed, 19 May 2021   Prob (F-statistic):               0.00
Time:                        17:36:25   Log-Likelihood:                -8423.4
No. Observations:               19036   AIC:                         1.687e+04
Df Residuals:                   19024   BIC:                         1.697e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.0182 

In [10]:
glm_modelx1 = smf.logit("default ~"+ '+'.join(X1), data = data).fit()
print(glm_modelx1.summary())

Optimization terminated successfully.
         Current function value: 0.443535
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                19036
Model:                          Logit   Df Residuals:                    19024
Method:                           MLE   Df Model:                           11
Date:                Wed, 19 May 2021   Pseudo R-squ.:                  0.1189
Time:                        17:36:25   Log-Likelihood:                -8443.1
converged:                       True   LL-Null:                       -9581.9
Covariance Type:            nonrobust   LLR p-value:                     0.000
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -3.2303      0.130    -24.861      0.000      -3.485      -2.976

In [11]:
####Categories!!!!
glm_modelx12 = sm.Logit(data[['default']], data[X1]).fit()
print(glm_modelx12.summary())

Optimization terminated successfully.
         Current function value: 0.493170
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                19036
Model:                          Logit   Df Residuals:                    19031
Method:                           MLE   Df Model:                            4
Date:                Wed, 19 May 2021   Pseudo R-squ.:                 0.02024
Time:                        17:36:25   Log-Likelihood:                -9388.0
converged:                       True   LL-Null:                       -9581.9
Covariance Type:            nonrobust   LLR p-value:                 1.124e-82
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
sales_mil_log            0.1513      0.025      5.992      0.000       0.102       0.201

In [12]:
# Check model X2
glm_modelx2 = smf.logit("default ~"+ '+'.join(X2), data = data).fit()
print(glm_modelx2.summary())

Optimization terminated successfully.
         Current function value: 0.423258
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                19036
Model:                          Logit   Df Residuals:                    19017
Method:                           MLE   Df Model:                           18
Date:                Wed, 19 May 2021   Pseudo R-squ.:                  0.1591
Time:                        17:36:25   Log-Likelihood:                -8057.1
converged:                       True   LL-Null:                       -9581.9
Covariance Type:            nonrobust   LLR p-value:                     0.000
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  -2.3506      0.148    -15.877      0.000      -2.641   

In [13]:
#calculate average marginal effects (dy/dx) for logit

In [14]:
### !!!!!!!!!!!!!small differences for categorical variables!!!!!!!!!!!!!!!!!!!!

mx2 = glm_modelx2.get_margeff(dummy=False)
print(mx2.summary()) 

        Logit Marginal Effects       
Dep. Variable:                default
Method:                          dydx
At:                           overall
                             dy/dx    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
ind2_cat[T.27.0]            0.0195      0.024      0.829      0.407      -0.027       0.066
ind2_cat[T.28.0]            0.0044      0.018      0.238      0.812      -0.032       0.040
ind2_cat[T.29.0]            0.0545      0.030      1.816      0.069      -0.004       0.113
ind2_cat[T.30.0]           -0.0044      0.039     -0.113      0.910      -0.082       0.073
ind2_cat[T.33.0]           -0.0212      0.018     -1.161      0.245      -0.057       0.015
ind2_cat[T.55.0]            0.0173      0.018      0.985      0.324      -0.017       0.052
ind2_cat[T.56.0]            0.0622      0.015      4.068      0.000       0.032       0.092
sales_mil_log       

In [15]:
################

#sum_table <- summary(glm_modelx2) %>%
#  coef() %>%
#  as.data.frame() %>%
#  select(Estimate) %>%
#  mutate(factor = row.names(.)) %>%
#  merge(summary(mx2)[,c("factor","AME")])

#kable(x = sum_table, format = "latex", digits = 3,
#      col.names = c("Variable", "Coefficient", "dx/dy"),
#      caption = "Average Marginal Effects (dy/dx) for Logit Model") %>%
#  cat(.,file= paste0(output,"AME_logit_X2.tex"))

####################

In [16]:
df1 = pd.DataFrame.from_records(glm_modelx2.summary().tables[1].data)
header1 = df1.iloc[0] # grab the first row for the header
header1[0]='factor'
header1[1] = 'Estimate'
df1.columns = header1
df1 = df1[2:] # take the data less the header row and Intercept
df1.set_index('factor', inplace=True)

df2 = pd.DataFrame.from_records(mx2.summary().tables[1].data)
header2 = df2.iloc[0] # grab the first row for the header
header2[0]='factor'
header2[1] = 'AME'
df2.columns = header2
df2 = df2[1:] # take the data less the header row
df2.set_index('factor', inplace=True)

sum_table = pd.concat([df1, df2], axis=1)[['Estimate','AME']]

print (sum_table.to_latex(header=["Coefficient", "dx/dy"]))
########## TO DO #############

\begin{tabular}{lll}
\toprule
0 & Coefficient &       dx/dy \\
factor                  &             &             \\
\midrule
ind2\_cat[T.27.0]        &      0.1455 &      0.0195 \\
ind2\_cat[T.28.0]        &      0.0327 &      0.0044 \\
ind2\_cat[T.29.0]        &      0.4063 &      0.0545 \\
ind2\_cat[T.30.0]        &     -0.0330 &     -0.0044 \\
ind2\_cat[T.33.0]        &     -0.1578 &     -0.0212 \\
ind2\_cat[T.55.0]        &      0.1287 &      0.0173 \\
ind2\_cat[T.56.0]        &      0.4636 &      0.0622 \\
sales\_mil\_log           &     -0.1803 &     -0.0242 \\
sales\_mil\_log\_sq        &      0.0153 &      0.0021 \\
d1\_sales\_mil\_log\_mod    &     -0.4821 &     -0.0646 \\
profit\_loss\_year\_pl     &     -0.4501 &     -0.0603 \\
fixed\_assets\_bs         &     -0.8110 &     -0.1087 \\
share\_eq\_bs             &     -0.3881 &     -0.0520 \\
curr\_liab\_bs            &      0.1712 &      0.0229 \\
curr\_liab\_bs\_flag\_high  &      0.1354 &      0.0181 \\
curr\_liab\_bs\_fla

In [17]:
# baseline model is X4 (all vars, but no interactions) -------------------------------------------------------

In [18]:
ols_model = smf.ols("default ~"+ '+'.join(X4), data = data).fit()
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:                default   R-squared:                       0.192
Model:                            OLS   Adj. R-squared:                  0.189
Method:                 Least Squares   F-statistic:                     57.15
Date:                Wed, 19 May 2021   Prob (F-statistic):               0.00
Time:                        17:36:26   Log-Likelihood:                -7610.8
No. Observations:               19036   AIC:                         1.538e+04
Df Residuals:                   18956   BIC:                         1.601e+04
Df Model:                          79                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

In [19]:
glm_model = smf.logit("default ~"+ '+'.join(X4), data = data).fit()
print(glm_model.summary())

         Current function value: 0.409571
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                19036
Model:                          Logit   Df Residuals:                    18956
Method:                           MLE   Df Model:                           79
Date:                Wed, 19 May 2021   Pseudo R-squ.:                  0.1863
Time:                        17:36:28   Log-Likelihood:                -7796.6
converged:                      False   LL-Null:                       -9581.9
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                         -2.4000      1.124     -2.135      0.033      -4.603      -0.197
ind2_

C:\dev\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [20]:
#calculate average marginal effects (dy/dx) for logit

#!!!!!!!!!!!!!!!!!!!!!small differences!!!!!!!!!!!!!!!!!!!!!!!!!!!
m = glm_model.get_margeff(dummy=False)
print(m.summary()) 

        Logit Marginal Effects       
Dep. Variable:                default
Method:                          dydx
At:                           overall
                                    dy/dx    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
ind2_cat[T.27.0]                   0.0115      0.023      0.502      0.616      -0.034       0.057
ind2_cat[T.28.0]                   0.0029      0.018      0.158      0.874      -0.032       0.038
ind2_cat[T.29.0]                   0.0505      0.030      1.707      0.088      -0.007       0.108
ind2_cat[T.30.0]                  -0.0065      0.038     -0.170      0.865      -0.082       0.069
ind2_cat[T.33.0]                  -0.0173      0.018     -0.970      0.332      -0.052       0.018
ind2_cat[T.55.0]                   0.0201      0.017      1.164      0.244      -0.014       0.054
ind2_cat[T.56.0]                   0.0600      0.015    

In [21]:
df1 = pd.DataFrame.from_records(glm_model.summary().tables[1].data)
header1 = df1.iloc[0] # grab the first row for the header
header1[0]='factor'
header1[1] = 'Estimate'
header1[2] = 'Std. Error'
df1.columns = header1
df1 = df1[2:] # take the data less the header row and Intercept
df1.set_index('factor', inplace=True)

df2 = pd.DataFrame.from_records(m.summary().tables[1].data)
header2 = df2.iloc[0] # grab the first row for the header
header2[0]='factor'
header2[1] = 'AME'
df2.columns = header2
df2 = df2[1:] # take the data less the header row
df2.set_index('factor', inplace=True)

sum_table2 = pd.concat([df1, df2], axis=1)[['Estimate', 'Std. Error','AME']]
print (sum_table2.to_latex(header=["Coefficient", "SE", "dx/dy"]))
########## TO DO #############

\begin{tabular}{llll}
\toprule
0 & Coefficient &         SE &       dx/dy \\
factor                         &             &            &             \\
\midrule
ind2\_cat[T.27.0]               &      0.0896 &      0.179 &      0.0115 \\
ind2\_cat[T.28.0]               &      0.0222 &      0.140 &      0.0029 \\
ind2\_cat[T.29.0]               &      0.3920 &      0.230 &      0.0505 \\
ind2\_cat[T.30.0]               &     -0.0505 &      0.298 &     -0.0065 \\
ind2\_cat[T.33.0]               &     -0.1344 &      0.139 &     -0.0173 \\
ind2\_cat[T.55.0]               &      0.1565 &      0.134 &      0.0201 \\
ind2\_cat[T.56.0]               &      0.4663 &      0.118 &      0.0600 \\
m\_region\_loc[T.East]           &      0.0624 &      0.055 &      0.0080 \\
m\_region\_loc[T.West]           &     -0.1626 &      0.068 &     -0.0209 \\
urban\_m[T.2.0]                 &      0.0145 &      0.061 &      0.0019 \\
urban\_m[T.3.0]                 &     -0.0506 &      0.056 &     -0.0065 \\
s

In [ ]:
from sklearn.model_selection import train_test_split
X = data.drop('default',axis=1) 
y = data['default']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13505)

In [22]:
X_train.sort_index()[0:20]

train = data.sample(frac=0.8, random_state=13505)
test = data.loc[~data.index.isin(train.index)]
test.sort_index()[0:20]

In [35]:
random.seed(13505)
train_ids= random.sample(range(0, len(data.index)), round(0.8*len(data.index)))
test = data.loc[~data.index.isin(train_ids)]

In [34]:
test

,year,comp_id,begin,end,amort,curr_assets,curr_liab,extra_exp,extra_inc,extra_profit_loss,...,flag_miss_ceo_age,ceo_young,labor_avg_mod,flag_miss_labor_avg,default_f,sales_mil_log_sq,flag_low_d1_sales_mil_log,flag_high_d1_sales_mil_log,d1_sales_mil_log_mod,d1_sales_mil_log_mod_sq
2,2012,1.003200e+06,2012-01-01,2012-12-31,25.925926,2.222222e+01,1.099630e+04,0.000000,0.0,0.000000,...,1,0,0.621691,1,default,34.614876,0,0,-1.424773,2.029978
8,2012,1.046213e+06,2012-01-01,2012-12-31,66944.445312,2.406474e+06,1.225500e+06,19877.777344,0.0,-19877.777344,...,0,0,7.861111,0,no_default,2.837557,0,0,0.045760,0.002094
24,2012,2.293689e+06,2012-01-01,2012-12-31,381.481476,4.429629e+03,6.981481e+03,0.000000,0.0,0.000000,...,0,1,0.159722,0,no_default,7.887241,0,0,0.165765,0.027478
25,2012,2.371138e+06,2012-01-01,2012-12-31,0.000000,2.485185e+03,5.148148e+02,0.000000,0.0,0.000000,...,1,0,0.166667,0,no_default,24.155457,0,0,-0.032285,0.001042
31,2012,2.792794e+06,2012-01-01,2012-12-31,0.000000,2.048148e+03,1.740741e+02,0.000000,0.0,0.000000,...,0,1,0.621691,1,no_default,47.717083,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19023,2012,4.632899e+11,2012-01-01,2012-12-31,4100.000000,8.918519e+03,5.036667e+04,0.000000,0.0,0.000000,...,0,0,0.465278,0,no_default,3.600376,0,0,0.147610,0.021789
19028,2012,4.635078e+11,2012-01-01,2012-12-31,3925.926025,3.081481e+03,9.100000e+03,0.000000,0.0,0.000000,...,0,0,0.409722,0,no_default,11.609117,0,0,-0.195628,0.038270
19030,2012,4.638406e+11,2012-01-01,2012-12-31,800.000000,9.470370e+03,1.155556e+04,0.000000,0.0,0.000000,...,0,1,0.621691,1,no_default,14.107306,0,0,-0.393031,0.154473
19034,2012,4.640446e+11,2012-01-01,2012-12-31,76059.257812,4.776548e+06,1.297552e+06,0.000000,0.0,0.000000,...,1,0,1.840278,0,no_default,4.596972,0,0,0.294490,0.086725


In [32]:
train_ids

[0,
 1,
 3,
 4,
 5,
 6,
 7,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 26,
 27,
 28,
 29,
 30,
 32,
 33,
 34,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 44,
 45,
 46,
 49,
 50,
 52,
 55,
 59,
 60,
 61,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 72,
 73,
 74,
 75,
 77,
 78,
 79,
 80,
 82,
 83,
 84,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 94,
 96,
 97,
 99,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 114,
 116,
 117,
 118,
 120,
 121,
 122,
 123,
 125,
 126,
 127,
 129,
 130,
 131,
 132,
 134,
 135,
 137,
 138,
 139,
 140,
 141,
 143,
 144,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 167,
 168,
 169,
 171,
 172,
 173,
 174,
 175,
 176,
 178,
 179,
 180,
 183,
 184,
 186,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 219,
 220,
 221,
 222,
 223,